In [1]:
import pandas as pd

## Step 1: Read CSV files into DataFrames

In [2]:
met_df = pd.read_csv('./Met/met_metadata_final.csv')
semart_df = pd.read_csv('./SemArt/semart_metadata_final.csv')
rijksmuseum_df = pd.read_csv('./Rijksmuseum/rijksmuseum_metadata_final.csv')
ukiyoe_df = pd.read_csv('./Ukiyo-e/ukiyoe_metadata_final.csv', index_col=0)
wikiart_df = pd.read_csv('./Wikiart/wikiart_metadata_final.csv')
gac_df = pd.read_csv('./GAC/gac_metadata_final.csv')

/var/folders/pg/3l0tmhjs169g7ywv7zxbdghr0000gn/T/ipykernel_36244/3050932227.py:4: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  ukiyoe_df = pd.read_csv('./Ukiyo-e/ukiyoe_metadata_final.csv', index_col=0)


In [3]:
met_df

,accession_number,artist,carver,credit_line,culture,date,description,designer,dimensions,geography,keywords,maker,manufacturer,medium,met_id,retailer,title
0,10.125.452a,NaN,NaN,"Gift of Mrs. Russell Sage, 1909",NaN,1700–1800,NaN,NaN,H. 16 1/2 in. (41.9 cm),NaN,"Bells, Metal, Copper alloy, Bronze, Bell metal...",NaN,NaN,Bell metal,52,NaN,Andiron
1,1980.501.3,NaN,NaN,"Gift of Mr. and Mrs. Louis E. McFadden, 1980",American,1770–90,This beaker is one of three in the Museum’s co...,NaN,Overall: 4 in. (10.2 cm); 5 oz. 9 dwt. (170 g)...,"Made in New York, United States","Myers, Myer, Metal, Silver, Vessels, Beakers, ...",Myer Myers (1723–1795),NaN,Silver,431,NaN,Beaker
2,51.171.152,NaN,NaN,"Gift of Mrs. Charles W. Green, in memory of Dr...",American,1830–40,NaN,NaN,Diam. 9 1/2 in. (24.1 cm),"Made in New England, United States","Glass, Pressed glass, Lacy-pattern glass, Bowl...",NaN,NaN,Lacy pressed glass,762,NaN,Bowl
3,48.135.51,NaN,NaN,"Bequest of Adeline R. Brown, 1947",American or British,ca. 1750,NaN,NaN,H. 8 5/8 in. (21.9 cm),Possibly made in England; Possibly made in Uni...,"Metal, Copper alloy, Brass, Lighting devices, ...",NaN,NaN,Brass,1269,NaN,Candlestick
4,25.115.17,NaN,NaN,"Rogers Fund, 1925",American,1755–65,NaN,NaN,40 x 20 7/8 x 20 1/2 in. (101.6 x 53 x 52.1 cm),"Made in Philadelphia, Pennsylvania, United States","Wood, Walnut, Softwood, Pine, Furniture, Seati...",NaN,NaN,"Walnut, yellow pine",1584,NaN,Side Chair
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,11.60.147,NaN,NaN,"Bequest of Maria P. James, 1910",American,1810–20,NaN,NaN,29 x 45 x 36 3/4 in. (73.7 x 114.3 x 93.3 cm),"Made in New York, New York, United States","Metal, Copper alloy, Brass, Furniture, Tables,...",NaN,NaN,"Mahogany, gilt brass, tulip poplar, white pine",3454,NaN,Drop-leaf Pembroke Table
423,1978.414,NaN,NaN,"Purchase, Lenox, Incorporated Gift and Sansbur...",American,1792,NaN,NaN,H. 9 in. (22.9 cm),"Possibly made in Maryland, United States; Poss...","Glass, Blown glass, Engraving, Vessels, Bottle...",NaN,NaN,Blown and engraved glass,3711,NaN,Pocket bottle
424,35.105.2,NaN,NaN,"Rogers Fund, 1935",NaN,1700–1800,NaN,NaN,H. 2 1/2 in. (6.4 cm),NaN,"Metal, Iron and iron alloy, Iron, Arms, Firear...",NaN,NaN,Iron,4145,NaN,Gun Hook
425,83.2.18,NaN,NaN,"Gift of William H. Huntington, 1883",NaN,1864,NaN,NaN,Diam. 1 1/8 in. (2.9 cm),NaN,"Bolen, John A., Metal, Medals",John A. Bolen (1826–1906),NaN,Metal,4539,NaN,Medal


## Step 2: Rename overlapping fields

In [3]:
# met_df.rename(columns={
#     'description': 'description',
#     'artist': 'artist',
#     'title': 'title',
#     'date': 'date',
#     'medium': 'technique',
#     'type': 'type'
# }, inplace=True)

semart_df.rename(columns={
    'IMAGE_FILE': 'image_file',
    'DESCRIPTION': 'description',
    'AUTHOR': 'artist',
    'TITLE': 'title',
    'TECHNIQUE': 'technique',
    'DATE': 'date',
    'TYPE': 'type',
    'SCHOOL': 'school',
    'SPLIT': 'split',
    'TIMEFRAME': 'timeframe'
}, inplace=True)

rijksmuseum_df.rename(columns={
    'filename': 'image_file',
    'description': 'description',
    'creator': 'artist',
    'title': 'title',
    'date': 'date',
    'type': 'type'
}, inplace=True)

ukiyoe_df.rename(columns={
    'image_file': 'image_file',
    'description': 'description',
    'artistString': 'artist',
    'title': 'title',
    'date': 'date',
    'type': 'type'
}, inplace=True)
# Drop the 'Unnamed: 42' column
ukiyoe_df.drop('Unnamed: 42', axis=1, inplace=True)

wikiart_df.rename(columns={
    'description': 'description',
    'filename': 'image_file',
    'artist': 'artist',
    'title': 'title',
    'date': 'date',
    'genre': 'type'
}, inplace=True)

gac_df.rename(columns={
    'artwork_path': 'image_file',
    'main_text': 'description',
    'creator': 'artist',
    'title': 'title',
    'date': 'date',
    'type': 'type'
}, inplace=True)

## Step 3: Convert each DataFrame to XML 

In [4]:
import xml.etree.ElementTree as ET 

def dataframe_to_xml(df, root_name, row_name):
    """Convert a DataFrame to an XML string."""
    root = ET.Element(root_name)
    for i, row in df.iterrows():
        row_elem = ET.SubElement(root, row_name)
        for field, value in row.items():
            field_elem = ET.SubElement(row_elem, field)
            field_elem.text = str(value)
    return ET.tostring(root, encoding='unicode')
    
# Step 3: Convert each DataFrame to XML
# met_xml = dataframe_to_xml(met_df, 'Met', 'artwork')
semart_xml = dataframe_to_xml(semart_df, 'SemArt', 'artwork')
rijksmuseum_xml = dataframe_to_xml(rijksmuseum_df, 'Rijksmuseum', 'artwork')
ukiyoe_xml = dataframe_to_xml(ukiyoe_df, 'Ukiyo-e', 'artwork')
wikiart_xml = dataframe_to_xml(wikiart_df, 'WikiArt', 'artwork')
gac_df_xml = dataframe_to_xml(gac_df, 'GAC', 'artwork')

## Step 4: Step 4: Merge the XMLs into a hierarchical structure

In [7]:
datasets_root = ET.Element('Datasets')

# met_tree = ET.ElementTree(ET.fromstring(met_xml))
semart_tree = ET.ElementTree(ET.fromstring(semart_xml))
rijksmuseum_tree = ET.ElementTree(ET.fromstring(rijksmuseum_xml))
ukiyoe_tree = ET.ElementTree(ET.fromstring(ukiyoe_xml))
wikiart_tree = ET.ElementTree(ET.fromstring(wikiart_xml))
gac_tree = ET.ElementTree(ET.fromstring(gac_df_xml))

# datasets_root.append(met_tree.getroot())
datasets_root.append(semart_tree.getroot())
datasets_root.append(rijksmuseum_tree.getroot())
datasets_root.append(ukiyoe_tree.getroot())
datasets_root.append(wikiart_tree.getroot())
datasets_root.append(gac_tree.getroot())

## Step 5: Save the combined XML to a file

In [9]:
combined_tree = ET.ElementTree(datasets_root)
combined_tree.write('merged_datasets.xml', encoding='unicode')

print("Merged XML saved as 'merged_datasets.xml'.")

Merged XML saved as 'merged_datasets.xml'.
